In [12]:
from src.first_model.preprocessing import PreProcessingFirstModel
from src.first_model.train import Modeling

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import math

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
PATH_RAW = "/home/luba/Documents/DS/projects-courses-ongoing/bncc-classifier-[doing]/data/raw/bncc_first_classifier.csv"  # noqa
PATH_TO_SAVE = "/home/luba/Documents/DS/projects-courses-ongoing/bncc-classifier-[doing]/data/curated/bncc_first_classifier_cleaned.csv"  # noqa

In [3]:
prepro_obj = PreProcessingFirstModel(PATH_RAW)
frq_words, rare_words = prepro_obj.prepro()

# export list of frq and rare words on our data
# with open("frq_words.txt", "w") as f:
#     # print(frq_words)
#     _ = [f.write("%s\n" % item) for item in frq_words]

# with open("rare_words.txt", "w") as f:
#     # print(rare_words)
#     _ = [f.write("%s\n" % item) for item in rare_words]

# prepro_obj.export_cleaned_data(PATH_TO_SAVE)
modeling = Modeling(prepro_obj.data)
# # modeling.tokenization()
modeling.split_data()
# modeling.saving_pipeline()
# X_train_trans, X_test_trans = modeling.make_bag_of_words()
# modeling.train_evaluate_log_reg(
#     X_train_trans=X_train_trans, X_test_trans=X_test_trans
# )

In [10]:
modeling.X_train["questions_clean"]

71534    ['diagramas', 'pv', 'seguir', 'representam', '...
20097    ['programa', 'computador', 'compactar', 'arqui...
26713    ['querendo', 'pintar', 'nova', 'fachada', 'lem...
43723    ['ef05ma06no', 'final', 'ano', 'passado', 'dir...
18699                  ['encaminhar', 'foto', 'pag', '16']
                               ...                        
26940    ['alunos', 'observarem', 'diariamente', 'nasce...
28024    ['observe', 'charge', 'apresentada', 'seguir',...
95998    ['feisp', 'movimento', 'retilíneo', 'uniformem...
65525    ['circuito', 'dessa', 'questão', 'três', 'capa...
6785     ['cmdpii', 'read', 'text', 'picture', 'answer'...
Name: questions_clean, Length: 57335, dtype: object

In [14]:
# baselines
models = {
    "rg_lg": LogisticRegression(max_iter=500, n_jobs=6),
    "d_tree": DecisionTreeClassifier(max_depth=10),
    "r_forest": RandomForestClassifier(max_depth=500, n_jobs=6),
    "lgbm": LGBMClassifier(n_jobs=6),
}

In [17]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )

        count_vectorizer = CountVectorizer()
        count_vectorizer.fit(x_train)
        X_train_cv = count_vectorizer.transform(x_train)
        X_val_cv = count_vectorizer.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print(f1)

0.7278253142975671
0.19407828326202928
0.6550434106278179
0.7145164707089338
0.7336190154838125
0.19210636960698546
0.649369269371921
0.7211956814872637
0.7330740839432095
0.19092967263959068
0.6498061041604263
0.7089284864570269
0.714314744373907
0.18948359277264393
0.6538362995130778
0.7173655668562345
0.7352967775129926
0.18483499341909196
0.655190162784483
0.7227555791757245
